# 트리의 앙상블

## 랜덤포레스트

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
# 랜덤 포레스트 분류기
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
# 교차 검증 (5-fold cross validation)
# return_train_score: True로 지정하면 validation set의 score 뿐만 아니라 train set의 score를 함께 반환 - 과대적합 파악 가능
scores = cross_validate(rf, train_input, train_target, n_jobs=-1, return_train_score=True)

# 평균 계산 (train score까지 출력 가능)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
# train
rf.fit(train_input, train_target)
# feature 중요도 출력
print(rf.feature_importances_)

# features = ['alcohol', 'sugar', 'pH']

[0.23167441 0.50039841 0.26792718]


In [4]:
# oob_score: True로 지정하면 각 결정 트리의 OOB 샘플로 평가한 score의 평균값 반환
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
# OOB score 출력
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리

In [5]:
from sklearn.ensemble import ExtraTreesClassifier
# 엑스트라 트리 분류기
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)

# 교차 검증 (5-fold cross validation)
scores = cross_validate(et, train_input, train_target, n_jobs=-1, return_train_score=True)

# 평균 계산
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


## 그레이디언트 부스팅

In [6]:
from sklearn.ensemble import GradientBoostingClassifier

# 그레이디언트 부스팅 분류기
gb = GradientBoostingClassifier(random_state=42)
# 교차 검증
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

# 평균 계산
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [7]:
# n_estimators: 결정 트리 개수
# learning_rate: 학습률 (기본값은 0.1)
# 결정 트리 개수와 학습률을 늘려 성능 향상
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


## 히스토그램 기반 부스팅

In [8]:
from sklearn.ensemble import HistGradientBoostingClassifier

# 히스토그램 기반 그레이디언트 부스팅
hgb = HistGradientBoostingClassifier(random_state=42)
# 교차 검증
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

# 평균 출력
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [9]:
from sklearn.inspection import permutation_importance

# train
hgb.fit(train_input, train_target)
# permutation_importance() 함수: feature 중요도 계산
# n_repeats 매개변수: 랜덤하게 섞을 횟수 지정(기본값은 5)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
# 반복해서 계산한 중요도의 평균 출력
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [10]:
# test set에서 feature 중요도 계산
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [12]:
# 최종 성능
hgb.score(test_input, test_target)

0.8723076923076923

#### XGBoost

In [13]:
from xgboost import XGBClassifier

# XGBoost를 사용하여 HGB 모델 훈련(tree_method='hist')
xgb = XGBClassifier(tree_method='hist', random_state=42)
# 교차 검증
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

# 평균 출력
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


#### LightGBM

In [14]:
from lightgbm import LGBMClassifier

# LightGBM을 사용하여 HGB 모델 훈련
lgb = LGBMClassifier(random_state=42)
# 교차 검증
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

# 평균 출력
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
